In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from floris import TimeSeries
from floris.layout_visualization import plot_turbine_points
from hopp import ROOT_DIR
from hopp.utilities.keys import set_nrel_key_dot_env
from hopp.simulation import HoppInterface
from hopp.utilities.utilities import load_yaml
from hopp.tools.dispatch.plot_tools import (
    plot_battery_output, plot_battery_dispatch_error, plot_generation_profile
)

set_nrel_key_dot_env()

example_dir = ROOT_DIR.parent / "examples"
input_filepath = os.path.join(str(example_dir),"inputs","09-distributed-midsize.yaml")
hopp_config = load_yaml(input_filepath)